<div align="center"; span style="color:#336699"><b><h2>Minicurso pyForTraCC </h2></b></div>
<div align="center"; span style="color:#336699"><b><h3>3. Rastreamento de Mudanças Antropogênicas com Dados do MapBiomas</h3></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
   <sup>Autor: <a href="https://www.linkedin.com/in/helvecio-leal/"> Helvécio B. Leal Neto, <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><t>&nbsp;</t> 
   <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="mailto:fortracc.project@inpe.br">fortracc.project@inpe.br</a>
    <br/><br/>
    Última Atualização: Set 8, 2025
</div>

<br/>

<div style="text-align: center; margin: 0 auto; width: 80ch;">
<b>Resumo.</b> Este notebook apresenta uma aplicação do <a href="https://github.com/fortracc/pyfortracc">pyfortracc</a> para rastreamento de mudanças no uso e cobertura da terra usando dados do MapBiomas. O tutorial demonstra como aplicar algoritmos de rastreamento tradicionalmente usados para sistemas meteorológicos na análise de expansão de áreas antropogênicas ao longo de décadas. Inclui download automático de dados do MapBiomas, processamento de classificações de uso da terra, configuração de parâmetros específicos para rastreamento temporal de longo prazo, e visualizações que mostram a evolução espacial de atividades humanas - fornecendo uma ferramenta poderosa para estudos de mudanças ambientais e planejamento territorial.
</div>

<div align="center">

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fortracc/pyfortracc/blob/main/examples/WORCAP-Minicourse/3_Antropogenic_Tracking/3_Antropogenic_Tracking.ipynb) 

</div>
<br/>

### Roteiro
 [1. Instalação de Dependências](#install)<br>
 [2. Download de Dados do MapBiomas](#download)<br>
 [3. Função de Leitura e Processamento](#data)<br>
 [4. Configuração de Parâmetros](#namelist)<br>
 [5. Execução do Rastreamento](#track)<br>
 [6. Análise da Tabela de Rastreamento](#tracktable)<br>
 [7. Análise de Crescimento](#growth-analysis)<br>
 [8. Visualização Temporal](#visualization)<br>
 [9. Conclusão](#conclusion)<br>

#### 1. **Instalação de Dependências** <a class="anchor" id="install"></a>

Para este exemplo, utilizaremos dados raster do MapBiomas, que requerem a biblioteca GDAL para processamento eficiente. O GDAL (Geospatial Data Abstraction Library) é uma biblioteca fundamental para manipulação de dados geoespaciais.

**Importante**: A instalação do GDAL pode levar alguns minutos dependendo do sistema.

In [ ]:
# Instala o GDAL
!sudo apt install gdal-bin -y > /dev/null 2>&1 && echo "✅ gdal instalado com sucesso!" || echo "❌ Erro na instalação do gdal"

Além disso, instalaremos o `pyfortracc` para realizar o rastreamento de mudanças.

In [ ]:
# Comando para instalação da ultima versão estável do pyfortracc
!python -m pip install -qqq -U pyfortracc > /dev/null 2>&1 && echo "✅ pyfortracc instalado com sucesso!" || echo "❌ Erro na instalação"

#### 2. **Download de Dados do MapBiomas** <a class="anchor" id="download"></a>

O [MapBiomas](https://mapbiomas.org/) é uma iniciativa que mapeia anualmente a cobertura e uso da terra no Brasil desde 1985. Para este tutorial, vamos baixar dados de uma região específica da Amazônia e processar as classificações de uso da terra.

A função abaixo utiliza o GDAL para:
- **Baixar dados diretamente da nuvem**: Acesso aos dados do MapBiomas no Google Cloud Storage
- **Recortar área de interesse**: Definir uma bounding box específica
- **Redimensionar dados**: Padronizar resolução para 1024x1024 pixels
- **Processamento paralelo**: Acelerar download de múltiplos anos

**Configuração da Área de Estudo**:
- **Região**: Área na Amazônia (Pará)
- **Período**: 1985-2022 (38 anos de dados)
- **Resolução**: Aproximadamente 30 metros por pixel

In [ ]:
import pathlib
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import subprocess

def download_mapbiomas(url_template: str, bbox: str, start_year: int, end_year: int, max_workers: int = 4):
    # Cria o diretório de saída se não existir
    pathlib.Path('input').mkdir(parents=True, exist_ok=True)
    # Função interna para baixar e processar um único ano
    def _download(year: int):
        url = url_template.format(year)
        out_file = f"input/{year}.tif"
        gdal_command = [
            "gdal_translate",
            f"/vsicurl/{url}",
            out_file,
            "-b", "1",
            "-projwin", *bbox.split(),
            "-of", "GTiff",
            "-outsize", "1024", "1024",
        ]
        subprocess.run(gdal_command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        return year
    # Lista de anos a serem processados
    years = list(range(start_year, end_year + 1))
    # Uso de ThreadPoolExecutor para downloads paralelos
    with tqdm(total=len(years), desc="Downloading & Processing") as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(_download, y): y for y in years}
            for future in as_completed(futures):
                try:
                    _ = future.result()
                except Exception as e:
                    print(f"Erro no ano {futures[future]}: {e}")
                pbar.update(1)

**Executando o Download**:

O processo abaixo baixa os dados do MapBiomas para o período 1985-2022. Os parâmetros definidos são:

- **URL template**: Padrão de URL do MapBiomas com placeholder para o ano
- **Bounding box**: `-55 -3.54 -54 -4.46` (oeste, norte, leste, sul)
- **Workers**: 10 downloads paralelos para acelerar o processo

**Importante**: O download pode levar entre 5-15 minutos dependendo da conexão com a internet.

In [ ]:
# Download e processa os dados do MapBiomas para a área e período especificados
url = "https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_8/lclu/coverage/brasil_coverage_{}.tif"
bbox = "-55 -3.54 -54 -4.46"  # lonmin latmax lonmax latmin
download_mapbiomas(url, bbox, 1985, 2022, max_workers=10)

Após o download, os dados serão armazenados na pasta `input`. E em seguida importamos a biblioteca `pyfortracc` para o rastreamento.

In [ ]:
# Importa o pyfortracc e mostra a versão instalada
import pyfortracc
print('pyFortracc version', pyfortracc.__version__)

#### 3. **Função de Leitura e Processamento** <a class="anchor" id="data"></a>

A função `read_function` é responsável por ler e processar os dados do MapBiomas. Diferentemente dos exemplos anteriores que trabalhavam com dados meteorológicos, aqui processamos classificações de uso da terra.

**Processo de Classificação**:

1. **Leitura do raster**: Carrega dados de classificação de uso da terra
2. **Inversão vertical**: Corrige orientação dos dados para padrão geográfico
3. **Máscara antropogênica**: Identifica áreas de atividade humana baseada na [legenda do MapBiomas](https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/EN__Codigos_da_legenda_Colecao_7.pdf)

**Classes Antropogênicas Incluídas**:
- **Agricultura**: Culturas temporárias e permanentes
- **Pecuária**: Pastagens e áreas de criação
- **Urbano**: Áreas urbanizadas e infraestrutura
- **Mineração**: Atividades de extração mineral
- **Outros**: Outras atividades humanas identificadas pelo MapBiomas

In [ ]:
import rasterio
import numpy as np

def read_function(path):
	# Lê o arquivo raster
	with rasterio.open(path) as src:
		# Lê a banda 1 e inverte a matriz verticalmente
		data = src.read(1)[::-1]
	# Mascara antropogênica segundo a legenda do MapBiomas https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/EN__Codigos_da_legenda_Colecao_7.pdf
	antropogenic_class = [14,15,18,19,39,20,40,62,41,36,46,47,48,9,21]
	# Cria uma máscara binária: 1 para áreas antropogênicas, 0 para outras
	antropogenic_data = np.where(np.isin(data, antropogenic_class), 1, 0)
	return antropogenic_data

**Teste da Função de Leitura**:

Vamos testar a função de leitura com os dados de 1985 para verificar se o processamento está funcionando corretamente. O resultado deve ser uma matriz binária onde:
- **1**: Representa áreas antropogênicas (atividade humana)
- **0**: Representa áreas naturais (floresta, água, etc.)

In [ ]:
read_function('input/1985.tif')

#### 4. **Configuração de Parâmetros** <a class="anchor" id="namelist"></a>

Para rastreamento de mudanças antropogênicas, precisamos ajustar os parâmetros do pyfortracc para escalas temporais muito maiores que as utilizadas em meteorologia. Enquanto sistemas meteorológicos evoluem em horas/dias, mudanças no uso da terra ocorrem ao longo de anos/décadas.

**Parâmetros Específicos para Uso da Terra**:

- **Escala temporal**: 1 ano entre observações (vs. 10 minutos em meteorologia)
- **Delta time**: 365 dias (vs. minutos em exemplos meteorológicos)
- **Tolerância temporal**: 1 dia para acomodar variações de processamento
- **Sobreposição mínima**: 5% para conectar áreas que se expandem gradualmente
- **Tamanho mínimo**: 5 pixels para filtrar ruído de classificação

**Método de Clusterização**:
- **ndimage**: Otimizado para dados raster de uso da terra
- **Operador ==**: Detecta presença de atividade antropogênica (valor = 1)

In [ ]:
# Configurações para o rastreamento
name_list = {}
name_list['input_path'] = 'input/' # Diretório de entrada
name_list['output_path'] = 'output/' # Diretório de saída
name_list['thresholds'] = [1] # Limiares para detecção
name_list['min_cluster_size'] = [5] # Tamanho mínimo dos clusters
name_list['operator'] = '==' # Operador para comparação
name_list['timestamp_pattern'] = '%Y.tif' # Padrão de timestamp nos arquivos
name_list['delta_time'] = 60 * 24 * 365 # Delta de tempo máximo entre detecções (em minutos)
name_list['delta_tolerance'] = 60 * 24 # Tolerância para delta de tempo + 1 dia (em minutos)
name_list['min_overlap'] = 5 # Sobreposição mínima entre detecções (em porcentagem)
name_list['cluster_method'] = 'ndimage' # Método de clusterização
name_list['opt_correction'] = True # Correção opcional
name_list['validation'] = True # Aplica validação

## -- CONFIGURAÇÕES DE DIMENSÕES --
name_list['lon_min'] = -55.0 # Limites geográficos
name_list['lon_max'] = -54.0 # Limites geográficos
name_list['lat_min'] = -4.46 # Limites geográficos
name_list['lat_max'] = -3.54 # Limites geográficos

#### 5. **Execução do Rastreamento** <a class="anchor" id="track"></a>

Agora executamos o rastreamento das mudanças antropogênicas ao longo de 38 anos (1985-2022). Este processo irá:

1. **Detectar áreas antropogênicas** em cada ano
2. **Conectar áreas relacionadas** entre anos consecutivos
3. **Rastrear expansão/contração** de atividades humanas
4. **Identificar novos focos** de desenvolvimento
5. **Gerar trajetórias temporais** de mudanças no uso da terra

**Importante**: Este processamento pode levar vários minutos devido ao volume de dados (38 anos × 1024² pixels).

In [ ]:
# Executa o rastreamento com as configurações definidas
pyfortracc.track(name_list, read_function, parallel=True)

#### 6. **Análise da Tabela de Rastreamento** <a class="anchor" id="tracktable"></a>

A tabela de rastreamento contém informações detalhadas sobre cada área antropogênica rastreada ao longo do tempo. Para análises de uso da terra, as colunas mais relevantes são:

- **`uid`**: Identificador único de cada área antropogênica rastreada
- **`timestamp`**: Ano da observação
- **`size`**: Tamanho da área em pixels
- **`geometry`**: Contorno geográfico da área
- **`lifetime`**: Duração total de atividade (em anos)
- **`status`**: Status da área (NEW=nova, CON=contínua, SPL=divisão, MRG=fusão)

Vamos carregar e examinar os resultados do rastreamento:

In [ ]:
import pandas as pd
import glob
tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)
display(tracking_table.tail())

#### 7. **Análise de Crescimento** <a class="anchor" id="growth-analysis"></a>

Uma análise fundamental em estudos de uso da terra é identificar as áreas que mais se expandiram ao longo do tempo. Vamos identificar os 5 maiores clusters antropogênicos por área máxima atingida durante todo o período de estudo.

Esta análise nos permite identificar:
- **Grandes empreendimentos**: Fazendas, projetos de colonização, áreas urbanas
- **Padrões de expansão**: Como e quando as áreas se desenvolveram
- **Hotspots de desmatamento**: Regiões de intensa atividade antropogênica

In [ ]:
# Get top 5 largest size of all time
size = tracking_table.groupby('uid')['size'].max().to_frame()
size = size.sort_values(by='size', ascending=False)
size.head(5)

**Evolução Temporal das Maiores Áreas**:

O gráfico abaixo mostra a evolução do tamanho (em pixels) dos 3 maiores clusters antropogênicos ao longo do tempo. Cada linha representa uma área diferente, permitindo observar:

- **Padrões de crescimento**: Expansão gradual vs. explosiva
- **Períodos de intensificação**: Anos com maior atividade
- **Estabilização**: Quando as áreas atingem tamanho máximo
- **Declínio**: Possível abandono ou mudança de uso

In [ ]:
uid_list = size.index[0:3].tolist()
tracking_table.loc[tracking_table['uid'].isin(uid_list)].groupby('uid')['size'].plot(legend=True, 
                                                                                     title='Area of the 10 largest clusters',
                                                                                     xlabel='Timestamp', 
                                                                                     ylabel='Number of pixels');

#### 8. **Visualização Temporal** <a class="anchor" id="visualization"></a>

A animação temporal é uma ferramenta poderosa para visualizar mudanças no uso da terra ao longo de décadas. Diferentemente das animações meteorológicas que mostram movimento em horas, aqui observamos transformações que ocorrem ao longo de anos.

**Configurações de Visualização**:

- **Período**: 1985-2022 (38 anos de transformação)
- **Foco**: Os 3 maiores clusters identificados
- **Background**: Imagem de satélite para contexto geográfico
- **Contornos**: Destaque em vermelho para as áreas antropogênicas
- **Sem trajetórias**: Para uso da terra, o movimento não é relevante

**Interpretação**:
- **Vermelho**: Áreas de atividade antropogênica
- **Expansão**: Crescimento das áreas vermelhas ao longo dos anos
- **Fragmentação**: Divisão de grandes áreas em menores
- **Conectividade**: Fusão de áreas próximas

**Importante**: A geração da animação pode levar vários minutos devido ao processamento de 38 frames de alta resolução.

In [ ]:
# Cria animação do rastreamento
print("Gerando animação do rastreamento...")

# Chama a função de plotagem de animação
pyfortracc.plot_animation(
    # === CONFIGURAÇÕES BÁSICAS ===
    read_function=read_function,                    # Função de leitura dos dados
    figsize=(8, 6),                               # Tamanho da figura (largura, altura)
    name_list=name_list,                           # Dicionário com parâmetros
    
    # === CONFIGURAÇÕES TEMPORAIS ===
    start_timestamp="1985",  # Timestamp inicial
    end_timestamp="2022",    # Timestamp final

    # === CONFIGURAÇÕES DE VISUALIZAÇÃO ===
    info=False,                           # Não mostra nome das colunas de info
    trajectory=False,                        # Não mostra trajetórias
    centroid=False,                          # Não mostra centroides
    title='Área Antropizada com conectividade de pixel (1985-2022)', # Título da barra de cores
    max_val=1,
    min_val=1,
    nan_value=0,                            # Valor para dados NaN
    cbar=False,

    uid_list=uid_list,                     # Lista de UIDs para destacar (None para todos)
        
    # === CONFIGURAÇÕES DOS CONTORNOS E INFORMAÇÕES ===
    bound_color='red',                            # Cor dos contornos dos sistemas
    bound_linewidth=0.5,                           # Espessura da linha dos contornos

    background='satellite',                  # Imagem de fundo (google, satellite, terrain, hybrid, none)
    
    # === CONFIGURAÇÕES DE PERFORMANCE ===
    parallel=True                                 # Processamento serial (mais estável)
)

#### 9. **Conclusão** <a class="anchor" id="conclusion"></a>

**Parabéns!** Você concluiu com sucesso o terceiro e último exemplo do minicurso pyfortracc. Neste tutorial inovador, você aprendeu:

- **Aplicação interdisciplinar**: Como usar algoritmos de rastreamento meteorológico para análise de uso da terra
- **Processamento de dados do MapBiomas**: Download e manipulação de séries temporais de classificação
- **Configuração para escalas temporais longas**: Adaptação de parâmetros para mudanças que ocorrem ao longo de décadas
- **Análise de mudanças antropogênicas**: Identificação e quantificação de expansão de atividades humanas
- **Visualização de transformações territoriais**: Criação de animações mostrando evolução do uso da terra
- **Análise de padrões temporais**: Compreensão de dinâmicas de crescimento e desenvolvimento

### **Aplicações Potenciais**

Este método pode ser aplicado em diversos contextos:

1. **Monitoramento de Desmatamento**: Rastreamento de perda florestal ao longo do tempo
2. **Expansão Urbana**: Análise do crescimento de cidades e ocupação do território
3. **Desenvolvimento Agrícola**: Acompanhamento da expansão da fronteira agrícola
4. **Planejamento Territorial**: Suporte a políticas de ordenamento do território
5. **Estudos de Impacto Ambiental**: Quantificação de pressões antrópicas sobre ecossistemas
6. **Análise de Políticas Públicas**: Avaliação de efetividade de medidas de conservação

### **Limitações e Considerações**

- **Resolução temporal**: Limitada pela disponibilidade anual do MapBiomas
- **Resolução espacial**: Dependente da resolução dos dados de entrada (30m)
- **Classificação**: Sujeita às limitações e erros do MapBiomas
- **Conectividade**: Parâmetros de sobreposição devem ser ajustados por região

### **Recursos Adicionais**

- [Documentação do pyfortracc](https://fortracc.github.io/pyfortracc/)
- [Portal do MapBiomas](https://mapbiomas.org/)
- [Coleção 8 do MapBiomas](https://brasil.mapbiomas.org/en/statistics/)
- [Artigos sobre o algoritmo ForTraCC](https://doi.org/10.3390/rs14215408)
- [Suporte técnico](mailto:fortracc.project@inpe.br)